In [1218]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.plotly as ply
from datetime import date
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from collections import Counter
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning:

pylab import has clobbered these variables: ['test']
`%matplotlib` prevents importing * from pylab and numpy



In [407]:
disputes = pd.read_csv("/Users/danielgilberg/data_science/metis/practice/mcnulty/data/MID-level/MIDA_4.01.csv")
dispute_part = pd.read_csv("/Users/danielgilberg/data_science/metis/practice/mcnulty/data/MID-level/MIDB_4.01.csv")

In [115]:
mask = (disputes.HostLev>=3 ) & (disputes.Fatality > 0) 
major_events = disputes[mask]

In [116]:
military_df = pd.read_csv("/Users/danielgilberg/data_science/metis/practice/mcnulty/data/NMC_5_0/NMC_5_0.csv")

In [117]:
military_df = military_df.rename(columns={"year": "StYear", "version":"Version"})

In [118]:
participant_info = pd.merge(dispute_part, military_df, on=["ccode", "StYear"])

In [119]:
participant_info.columns

Index(['DispNum3', 'DispNum4', 'StAbb', 'ccode', 'StDay', 'StMon', 'StYear',
       'EndDay', 'EndMon', 'EndYear', 'SideA', 'RevState', 'RevType1',
       'RevType2', 'Fatality', 'FataPre', 'HiAct', 'HostLev', 'Orig',
       'Version_x', 'stateabb', 'milex', 'milper', 'irst', 'pec', 'tpop',
       'upop', 'cinc', 'Version_y'],
      dtype='object')

In [120]:
participant_info = participant_info.drop(["Version_x", "Version_y"], axis=1)
participant_info.columns

Index(['DispNum3', 'DispNum4', 'StAbb', 'ccode', 'StDay', 'StMon', 'StYear',
       'EndDay', 'EndMon', 'EndYear', 'SideA', 'RevState', 'RevType1',
       'RevType2', 'Fatality', 'FataPre', 'HiAct', 'HostLev', 'Orig',
       'stateabb', 'milex', 'milper', 'irst', 'pec', 'tpop', 'upop', 'cinc'],
      dtype='object')

In [127]:
dispute_side_info = participant_info.groupby(["DispNum3", "DispNum4"], as_index=False).agg({"milex": sum, 
                                                                                "milper": sum, "irst": sum,
                                                                                "tpop": sum, "upop":sum})

In [122]:
def is_major_event(fatality, hostility):
    if fatality > 0 and hostility > 2:
        return 1
    else:
        return 0
disputes["Major_Dispute"] = disputes.apply(lambda row: is_major_event(row["Fatality"], row["HostLev"]), axis=1)

In [331]:
all_sides = pd.merge(disputes, dispute_side_info, on=["DispNum3", "DispNum4"])


In [332]:
trade_df = pd.read_csv("/Users/danielgilberg/data_science/metis/practice/mcnulty/data/COW_Trade_4.0/National_COW_4.0.csv")

In [404]:
# trade_df = trade_df[["ccode", "statename", "stateabb", "year", "imports", "exports"]]
# trade_df = trade_df.rename(columns={"year": "StYear"})
# df = pd.merge(participant_info, trade_df, on=["ccode", "StYear"])
trade_df = df.groupby(["ccode", "StYear"], as_index=False).agg({"imports": sum, "exports": sum})

In [412]:
disputes_with_trade = dispute_part.merge(trade_df, on=["ccode", "StYear"])

In [420]:
total_trade = disputes_with_trade.groupby(["DispNum3", "DispNum4"], as_index=False).agg({"imports": sum, "exports": sum})

In [424]:
all_sides = pd.merge(all_sides, total_trade, on=["DispNum3", "DispNum4"])

In [743]:
all_sides["imports"] = all_sides.imports.fillna(0)
all_sides["exports"] = all_sides.exports.fillna(0)

In [907]:
side_count = disputes.groupby(["DispNum3", "DispNum4"], as_index=False).agg({"NumA": sum, "NumB": sum})
side_count["Country_Count"] = side_count.apply(lambda row: row["NumA"] + row["NumB"], axis=1)
side_count = side_count.drop(["NumA", "NumB"], axis=1)

In [909]:
all_sides = all_sides.merge(side_count, on=["DispNum3", "DispNum4"])

In [1123]:
revisionist_df = dispute_part.groupby(["DispNum3", "DispNum4"], as_index=False).agg({"RevState": sum})
all_sides = all_sides.merge(revisionist_df, on=["DispNum3", "DispNum4"])

In [1060]:
x = all_sides[all_sides.Major_Dispute ==1]
y = all_sides[all_sides.Major_Dispute ==0]

105.41351888667992

#### Modeling

In [1167]:
rfc = RandomForestClassifier(class_weight="balanced")

In [1170]:
y = all_sides["Major_Dispute"]
# X = all_sides[["milex", "milper", "irst", "tpop", "upop"]]
X = all_sides[["milper", "irst", "exports", "Country_Count", "Recip", "MaxDur", "milex"]]
len(disputes[disputes.HostLev==3])


milper             int64
irst               int64
exports          float64
Country_Count      int64
Recip              int64
MaxDur             int64
milex              int64
dtype: object

In [1309]:
rfc.fit(X,y)
recall = cross_val_score(rfc, X, y, cv=10, scoring='recall').mean()
precision = cross_val_score(rfc, X, y, cv=10, scoring='precision').mean()
accuracy = cross_val_score(rfc, X, y, cv=10, scoring='accuracy').mean()
f1 = cross_val_score(rfc, X, y, cv=10, scoring='f1').mean()
y_pred = rfc.predict(X)
unique, counts = np.unique(y_pred, return_counts=True)
unique, counts, accuracy, precision
all_sides.columns

Index(['DispNum3', 'DispNum4', 'StDay', 'StMon', 'StYear', 'EndDay', 'EndMon',
       'EndYear', 'Outcome', 'Settle', 'Fatality', 'FatalPre', 'MaxDur',
       'MinDur', 'HiAct', 'HostLev', 'Recip', 'NumA', 'NumB', 'Link1', 'Link2',
       'Link3', 'Ongo2010', 'Version', 'Major_Dispute', 'milex', 'milper',
       'irst', 'tpop', 'upop', 'imports', 'exports', 'Country_Count',
       'RevState'],
      dtype='object')

In [433]:
logreg = LogisticRegression(class_weight="balanced")
logreg.fit(X, y)
y_pred = logreg.predict(X)
unique, counts = np.unique(y_pred, return_counts=True)
cross_val_score(logreg, X, y, cv=10, scoring='roc_auc').mean()

0.53077014096758524

In [483]:
major_disputes= all_sides[all_sides.Major_Dispute == 1]
len(major_disputes)

421

In [484]:
non_major_events = all_sides[all_sides.Major_Dispute == 0]
len(non_major_events)

2012

In [624]:
major_disp_x = major_disputes["Major_Dispute"]
major_disp_y = major_disputes[["irst", "milper", "upop", "imports", "exports"]]

In [625]:
non_major_x = non_major_events["Major_Dispute"]
non_major_y = non_major_events[["irst", "milper", "upop"]]

In [1221]:
df = all_sides[["Major_Dispute","milper", "irst", "exports","imports", "Country_Count", 
                "Recip", "MaxDur", "milex", "RevState"]]
train,test = train_test_split(df, test_size=.3, random_state=42, stratify=df["Major_Dispute"])
len(test), len(train)

(730, 1703)

In [1222]:
major_train = train[train.Major_Dispute == 1]
non_major_train = train[train.Major_Dispute == 0]

In [1201]:
len(major_train), len(non_major_train)

(295, 1408)

In [1296]:
# df_major_resampled = resample(major_train, replace=True, n_samples=1408, random_state=123)
# training_df = pd.concat([non_major_train, df_major_resampled])
df_non_major_resampled = resample(non_major_train, replace=False, n_samples=295, random_state=123)
training_df = pd.concat([df_non_major_resampled, major_train])
X_train = training_df[["milper", "irst", "Country_Count", "Recip", "milex", "exports", "RevState", "imports"]]
y_train = training_df["Major_Dispute"]
X_test = test[["milper", "irst", "Country_Count", "Recip", "milex", "exports", "RevState", "imports"]]
y_test = test["Major_Dispute"]

In [1362]:
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
print("Precision Score: {0:.3f}".format(metrics.precision_score(y_test, y_pred)))
print("Recall Score: {0:.3f}".format(metrics.recall_score(y_test, y_pred)))
#print(metrics.classification_report(y_test, y_pred))
Counter(y_pred)

Precision Score: 0.358
Recall Score: 0.683


Counter({0: 490, 1: 240})

recall--how many of the disputes I catch
precision--guessed disputes/total disputes

In [1248]:
y_pred = rfc.predict(X_test)

In [1250]:
metrics.recall_score(y_test, y_pred)

0.66666666666666663

In [1045]:
X = all_sides[["milper", "irst", "Country_Count", "milex", "Recip"]]
y = all_sides["Major_Dispute"]

In [1047]:
sv = SVC(class_weight="balanced")
logreg = LogisticRegression()
a = cross_val_score(rfc, X, y, cv=4, scoring='recall')
print(a)
print(a.mean())

[ 0.28301887  0.22857143  0.22857143  0.19047619]
0.232659478886


RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

SyntaxError: illegal target for annotation (<ipython-input-1363-1553e6cd93f0>, line 1)